In [7]:
import pandas as pd
import numpy as np
import os


In [ ]:
# Load Data
over_10_meds_data = pd.read_csv("input_data/pct_patients_on_over_10_meds.csv")
acb_burden_data = pd.read_csv("input_data/pct_patients_with_acb_burden_6_or_more.csv")
hypotensive_meds_data = pd.read_csv("input_data/pct_patients_with_3_hypotensive_risk_meds.csv")
over_10_meds_national_data = pd.read_csv("input_data/pct_patients_on_over_10_meds_national.csv")
acb_burden_national_data = pd.read_csv("input_data/pct_patients_with_acb_burden_6_or_more_national.csv")
hypotensive_meds_national_data = pd.read_csv("input_data/pct_patients_with_3_hypotensive_risk_meds_national.csv")

In [9]:
## CLEANING

# Strip out practices with (C or D) codes
over_10_meds_data = over_10_meds_data[~over_10_meds_data['Practice plus Code'].str.contains(r'\( ?[CD] ?\d', na=False)]
acb_burden_data = acb_burden_data[~acb_burden_data['Practice plus Code'].str.contains(r'\( ?[CD] ?\d', na=False)]
hypotensive_meds_data = hypotensive_meds_data[~hypotensive_meds_data['Practice plus Code'].str.contains(r'\( ?[CD] ?\d', na=False)]

# Convert 'Month' to datetime format
over_10_meds_data['Month'] = pd.to_datetime(over_10_meds_data['Month'], format='%b-%y')
acb_burden_data['Month'] = pd.to_datetime(acb_burden_data['Month'], format='%b-%y')
hypotensive_meds_data['Month'] = pd.to_datetime(hypotensive_meds_data['Month'], format='%b-%y')
over_10_meds_national_data['Month'] = pd.to_datetime(over_10_meds_national_data['Month'], format='%b-%y')
acb_burden_national_data['Month'] = pd.to_datetime(acb_burden_national_data['Month'], format='%b-%y')
hypotensive_meds_national_data['Month'] = pd.to_datetime(hypotensive_meds_national_data['Month'], format='%b-%y')

# Rename national 'Value' columns to match the column that will be calculated for each local dataset
over_10_meds_national_data = over_10_meds_national_data.rename(columns={
    'Value': 'Percentage patients >75 on 10 or more meds',
    'Country': 'Commissioner / Provider'
})
acb_burden_national_data = acb_burden_national_data.rename(columns={
    'Value': 'Percentage patients with ACB burden 6 or more',
    'Country': 'Commissioner / Provider'
})
hypotensive_meds_national_data = hypotensive_meds_national_data.rename(columns={
    'Value': 'Percentage patients with 3 hypotensive risk meds',
    'Country': 'Commissioner / Provider'
})

# Add mapping for local SICBL names
organisation_legend_mapping = {
    'NHS NORTH EAST AND NORTH CUMBRIA ICB - 84H': 'Durham',
    'NHS NORTH EAST AND NORTH CUMBRIA ICB - 00P': 'Sunderland',
    'NHS NORTH EAST AND NORTH CUMBRIA ICB - 00L': 'Northumberland',
    'NHS NORTH EAST AND NORTH CUMBRIA ICB - 01H': 'North Cumbria',
    'NHS NORTH EAST AND NORTH CUMBRIA ICB - 13T': 'Newcastle-Gateshead',
    'NHS NORTH EAST AND NORTH CUMBRIA ICB - 16C': 'Tees Valley',
    'NHS NORTH EAST AND NORTH CUMBRIA ICB - 99C': 'North Tyneside',
    'NHS NORTH EAST AND NORTH CUMBRIA ICB - 00N': 'South Tyneside',
    'ENGLAND': 'England'
}

# Replace long-SICBL names with local names using the mapping
over_10_meds_data['Commissioner / Provider'] = over_10_meds_data['Commissioner / Provider'].replace(organisation_legend_mapping)
acb_burden_data['Commissioner / Provider'] = acb_burden_data['Commissioner / Provider'].replace(organisation_legend_mapping)
hypotensive_meds_data['Commissioner / Provider'] = hypotensive_meds_data['Commissioner / Provider'].replace(organisation_legend_mapping)
over_10_meds_national_data['Commissioner / Provider'] = over_10_meds_national_data['Commissioner / Provider'].replace(organisation_legend_mapping)
acb_burden_national_data['Commissioner / Provider'] = acb_burden_national_data['Commissioner / Provider'].replace(organisation_legend_mapping)
hypotensive_meds_national_data['Commissioner / Provider'] = hypotensive_meds_national_data['Commissioner / Provider'].replace(organisation_legend_mapping)


# Drop irrelevant columns
over_10_meds_data = over_10_meds_data.drop(columns=['Practice plus Code', 'Comparator Description', 'Age Band', 'Value'], errors='ignore')
acb_burden_data = acb_burden_data.drop(columns=['Practice plus Code', 'Comparator Description', 'Age Band', 'Value'], errors='ignore')
hypotensive_meds_data = hypotensive_meds_data.drop(columns=['Practice plus Code', 'Comparator Description', 'Age Band', 'Value'], errors='ignore')
over_10_meds_national_data = over_10_meds_national_data.drop(columns=['Comparator Description', 'Age Band'], errors='ignore')
acb_burden_national_data = acb_burden_national_data.drop(columns=['Comparator Description', 'Age Band'], errors='ignore')
hypotensive_meds_national_data = hypotensive_meds_national_data.drop(columns=['Comparator Description', 'Age Band'], errors='ignore')


# Checkpoint output as cleaned CSVs
over_10_meds_data.to_csv("output_data/1_cleaned_over_10_meds.csv", index=False)
acb_burden_data.to_csv("output_data/1_cleaned_acb_burden.csv", index=False)
hypotensive_meds_data.to_csv("output_data/1_cleaned_hypotensive_meds.csv", index=False)
over_10_meds_national_data.to_csv("output_data/1_cleaned_over_10_meds_national.csv", index=False)
acb_burden_national_data.to_csv("output_data/1_cleaned_acb_burden_national.csv", index=False)
hypotensive_meds_national_data.to_csv("output_data/1_cleaned_hypotensive_meds_national.csv", index=False)



In [10]:
## AGGREGATION

# Aggregate by SICBL and Month
over_10_meds_data = over_10_meds_data.groupby(['Commissioner / Provider', 'Month'], as_index=False).sum()
acb_burden_data = acb_burden_data.groupby(['Commissioner / Provider', 'Month'], as_index=False).sum()
hypotensive_meds_data = hypotensive_meds_data.groupby(['Commissioner / Provider', 'Month'], as_index=False).sum()

# Checkpoint output as aggregated CSVs
over_10_meds_data.to_csv("output_data/2_aggregated_over_10_meds.csv", index=False)
acb_burden_data.to_csv("output_data/2_aggregated_acb_burden.csv", index=False)
hypotensive_meds_data.to_csv("output_data/2_aggregated_hypotensive_meds.csv", index=False)



In [11]:
## PROCESSING

# Create ICB summary rows
icb_rows = (
    over_10_meds_data
    .groupby('Month')[['Numerator', 'Denominator']]
    .sum()
    .reset_index()
)

# Label, reorder, and append ICB rows to the main dataframe
icb_rows['Commissioner / Provider'] = 'North East and North Cumbria'
icb_rows = icb_rows[['Commissioner / Provider', 'Month', 'Numerator', 'Denominator']]
over_10_meds_data = pd.concat([over_10_meds_data, icb_rows], ignore_index=True)

# Derive percentage values based on aggregated numerators / denominators
over_10_meds_data['Percentage patients >75 on 10 or more meds'] = np.where(
    over_10_meds_data['Denominator'] != 0,
    (over_10_meds_data['Numerator'] / over_10_meds_data['Denominator']) * 100,
    0
)


# Create ICB summary rows
icb_rows = (
    acb_burden_data
    .groupby('Month')[['Numerator', 'Denominator']]
    .sum()
    .reset_index()
)

# Label, reorder, and append ICB rows to the main dataframe
icb_rows['Commissioner / Provider'] = 'North East and North Cumbria'
icb_rows = icb_rows[['Commissioner / Provider', 'Month', 'Numerator', 'Denominator']]
acb_burden_data = pd.concat([acb_burden_data, icb_rows], ignore_index=True)

# Derive percentage values based on aggregated numerators / denominators
acb_burden_data['Percentage patients with ACB burden 6 or more'] = np.where(
    acb_burden_data['Denominator'] != 0,
    (acb_burden_data['Numerator'] / acb_burden_data['Denominator']) * 100,
    0
)


# Create ICB summary rows
icb_rows = (
    hypotensive_meds_data
    .groupby('Month')[['Numerator', 'Denominator']]
    .sum()
    .reset_index()
)

# Label, reorder, and append ICB rows to the main dataframe
icb_rows['Commissioner / Provider'] = 'North East and North Cumbria'
icb_rows = icb_rows[['Commissioner / Provider', 'Month', 'Numerator', 'Denominator']]
hypotensive_meds_data = pd.concat([hypotensive_meds_data, icb_rows], ignore_index=True)

# Derive percentage values based on aggregated numerators / denominators
hypotensive_meds_data['Percentage patients with 3 hypotensive risk meds'] = np.where(
    hypotensive_meds_data['Denominator'] != 0,
    (hypotensive_meds_data['Numerator'] / hypotensive_meds_data['Denominator']) * 100,
    0
)

# Append national row to each dataset
over_10_meds_data = pd.concat([over_10_meds_data, over_10_meds_national_data], ignore_index=True)
acb_burden_data = pd.concat([acb_burden_data, acb_burden_national_data], ignore_index=True)
hypotensive_meds_data = pd.concat([hypotensive_meds_data, hypotensive_meds_national_data], ignore_index=True)

# Round percentage values to 2 decimal places
over_10_meds_data['Percentage patients >75 on 10 or more meds'] = over_10_meds_data['Percentage patients >75 on 10 or more meds'].round(2)
acb_burden_data['Percentage patients with ACB burden 6 or more'] = acb_burden_data['Percentage patients with ACB burden 6 or more'].round(2)
hypotensive_meds_data['Percentage patients with 3 hypotensive risk meds'] = hypotensive_meds_data['Percentage patients with 3 hypotensive risk meds'].round(2)

# Checkpoint output as aggregated CSVs
over_10_meds_data.to_csv("output_data/3_processed_over_10_meds.csv", index=False)
acb_burden_data.to_csv("output_data/3_processed_acb_burden.csv", index=False)
hypotensive_meds_data.to_csv("output_data/3_processed_hypotensive_meds.csv", index=False)



In [13]:
# Merge all three datasets on Commissioner / Provider and Month
merged_data = over_10_meds_data.merge(
    acb_burden_data[['Commissioner / Provider', 'Month', 'Percentage patients with ACB burden 6 or more']],
    on=['Commissioner / Provider', 'Month'],
    how='outer'
).merge(
    hypotensive_meds_data[['Commissioner / Provider', 'Month', 'Percentage patients with 3 hypotensive risk meds']],
    on=['Commissioner / Provider', 'Month'],
    how='outer'
)

# Drop Numerator and Denominator if they persist (i.e. from over_10_meds_data)
merged_data = merged_data.drop(columns=['Numerator', 'Denominator'], errors='ignore')

# Sort by Month and Commissioner for clean output
merged_data = merged_data.sort_values(by=['Month', 'Commissioner / Provider'])


# 1. Pivot for "10+ meds"
pivot_over_10 = merged_data.pivot_table(
    index='Month',
    columns='Commissioner / Provider',
    values='Percentage patients >75 on 10 or more meds'
).reset_index()

# 2. Pivot for "ACB burden ≥6"
pivot_acb = merged_data.pivot_table(
    index='Month',
    columns='Commissioner / Provider',
    values='Percentage patients with ACB burden 6 or more'
).reset_index()

# 3. Pivot for "3+ hypotensive meds"
pivot_hypotensive = merged_data.pivot_table(
    index='Month',
    columns='Commissioner / Provider',
    values='Percentage patients with 3 hypotensive risk meds'
).reset_index()

# Reformat 'Month' back to 'Mon-YY' format for display purposes
for df in [pivot_over_10, pivot_acb, pivot_hypotensive]:
    df['Month'] = df['Month'].dt.strftime('%b-%y')

# Save all pivot tables to one Excel file, each in a separate sheet
with pd.ExcelWriter("output_data/overprescribing_charts.xlsx", engine="openpyxl") as writer:
    pivot_over_10.to_excel(writer, sheet_name="10+ Meds", index=False)
    pivot_acb.to_excel(writer, sheet_name="ACB Burden", index=False)
    pivot_hypotensive.to_excel(writer, sheet_name="Hypotensive Meds", index=False)

